In [1]:
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]

In [2]:
def display(obj):
    cls = type(obj)
    if cls is type:
        return '<class {}>'.format(obj.__name__)
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return '<{} object>'.format(cls_name(obj))

In [3]:
def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print('-> {}.__{}__({})'.format(cls_name(args[0]), name, pseudo_args))

In [4]:
class Overriding:  # <1>
    """a.k.a. data descriptor or enforced descriptor"""

    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)  # <2>

    def __set__(self, instance, value):
        print_args('set', self, instance, value)

In [5]:
class OverridingNoGet:  # <3>
    """an overriding descriptor without ``__get__``"""

    def __set__(self, instance, value):
        print_args('set', self, instance, value)

In [6]:
class NonOverriding:  # <4>
    """a.k.a. non-data or shadowable descriptor"""

    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)

In [7]:
class Managed:  # <5>
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()

    def spam(self):  # <6>
        print('-> Managed.spam({})'.format(display(self)))

#### Overriding descriptor (a.k.a. data descriptor or enforced descriptor):

In [8]:
>>> obj = Managed()  # <1>
>>> obj.over  # <2>

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [9]:
>>> Managed.over  # <3>

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [10]:
>>> obj.over = 7  # <4>

-> Overriding.__set__(<Overriding object>, <Managed object>, 7)


In [11]:
>>> obj.over  # <5>

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [12]:
>>> obj.__dict__['over'] = 8  # <6>
>>> vars(obj)  # <7>

{'over': 8}

In [13]:
>>> obj.over  # <8>

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


#### Overriding descriptor without ``__get__``:

In [14]:
>>> obj.over_no_get  # doctest: +ELLIPSIS

In [15]:
>>> Managed.over_no_get  # doctest: +ELLIPSIS

In [16]:
>>> obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [17]:
>>> obj.over_no_get  # doctest: +ELLIPSIS

In [18]:
>>> obj.__dict__['over_no_get'] = 9
>>> obj.over_no_get

9

In [19]:
>>> obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [20]:
>>> obj.over_no_get

9

#### Non-overriding descriptor (a.k.a. non-data descriptor or shadowable descriptor):

In [21]:
>>> obj = Managed()
>>> obj.non_over  # <1>

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [22]:
>>> obj.non_over = 7  # <2>
>>> obj.non_over  # <3>

7

In [23]:
>>> Managed.non_over  # <4>

-> NonOverriding.__get__(<NonOverriding object>, None, <class Managed>)


In [24]:
>>> del obj.non_over  # <5>
>>> obj.non_over  # <6>

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


#### No descriptor type survives being overwritten on the class itself:

In [25]:
>>> obj = Managed()  # <1>
>>> Managed.over = 1  # <2>
>>> Managed.over_no_get = 2
>>> Managed.non_over = 3
>>> obj.over, obj.over_no_get, obj.non_over  # <3>

(1, 2, 3)

#### Methods are non-overriding descriptors:

In [26]:
>>> obj.spam  # doctest: +ELLIPSIS

<bound method Managed.spam of <__main__.Managed object at 0x000002C3EB8B9FF0>>

In [27]:
>>> Managed.spam  # doctest: +ELLIPSIS

<function __main__.Managed.spam(self)>

In [28]:
>>> obj.spam()

-> Managed.spam(<Managed object>)


In [29]:
>>> Managed.spam()

TypeError: Managed.spam() missing 1 required positional argument: 'self'

In [30]:
>>> Managed.spam(obj)

-> Managed.spam(<Managed object>)


In [31]:
>>> Managed.spam.__get__(obj)  # doctest: +ELLIPSIS

<bound method Managed.spam of <__main__.Managed object at 0x000002C3EB8B9FF0>>

In [32]:
>>> obj.spam.__func__ is Managed.spam

True

In [33]:
>>> obj.spam = 7
>>> obj.spam

7

#### NOTE: These tests are here because I can't add callouts after +ELLIPSIS directives and if doctest runs them without +ELLIPSIS I get test failures.


In [34]:
>>> obj.over_no_get  # <1>

2

In [35]:
>>> Managed.over_no_get  # <2>

2

In [36]:
>>> obj.over_no_get = 7  # <3>

In [37]:
>>> obj.over_no_get  # <4>

7

In [38]:
>>> obj.__dict__['over_no_get'] = 9  # <5>
>>> obj.over_no_get  # <6>

9

In [39]:
>>> obj.over_no_get = 7  # <7>

In [40]:
>>> obj.over_no_get  # <8>

7

#### Methods are non-overriding descriptors:

In [41]:
>>> obj = Managed()
>>> obj.spam  # <1>

<bound method Managed.spam of <__main__.Managed object at 0x000002C3ECB3A950>>

In [42]:
>>> Managed.spam  # <2>

<function __main__.Managed.spam(self)>

In [43]:
>>> obj.spam = 7  # <3>
>>> obj.spam

7